In [32]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
import os
import shap
from GenerateSHAPFuncs import *
import matplotlib.pyplot as plt 
import xgboost
import re
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

#write versions
import sys
import pkg_resources

# Get the Python version
python_version = sys.version

# Get the list of installed packages with versions
installed_packages = sorted(["{}=={}".format(d.project_name, d.version) for d in pkg_resources.working_set])

# Define output file
output_file = "../Paper_Figures/Package_Info/pythonStep3_environment.txt"

# Write to file
with open(output_file, "w") as f:
    f.write(f"Python Version:\n{python_version}\n\n")
    f.write("Installed Packages:\n")
    f.write("\n".join(installed_packages))

print(f"Python environment details saved to {output_file}")

Python environment details saved to ../Paper_Figures/Package_Info/pythonStep3_environment.txt


In [34]:
# This code chunk will produce the FM dataset used in the paper. This chunk is self contained and does not need GenerateSHAPFuncs.py
import pandas as pd
import numpy as np
import os
import re
import shap
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

SEED = 123
np.random.seed(SEED)

# ---------------------------
# Helper: Clean Column Name
# ---------------------------
def clean_column_name(name):
    """
    Clean a column name by:
      - Converting to a string.
      - Replacing any non-alphanumeric character (except underscore) with an underscore.
      - Prefixing with 'f_' if the name starts with a digit.
      - Using a default name if empty.
    """
    name = str(name)
    # Replace non-alphanumeric characters (except underscore) with underscore
    name = re.sub(r'[^0-9a-zA-Z_]+', '_', name)
    if not name:
        name = "col"
    if name[0].isdigit():
        name = "f_" + name
    return name

# ---------------------------
# Function: Add Median Lifespan Increase
# ---------------------------
def add_median_lifespan_increase(data):
    data = data.copy()
    data['median_lifespan_increase'] = np.nan  # Initialize with NaN

    # Define mapping from 'Grp_Sex' to 'median_lifespan_increase'
    mapping = {
        "M_Rapa": 23,
        "M_Cana": 14,
        "M_17aE2": 12,
        "M_CR": 32,
        "M_Aca": 22,
        "M_Cont_12": 0
    }
    data['median_lifespan_increase'] = data['Grp_Sex'].map(mapping)
    return data

# ---------------------------
# Function: Train Model & Compute SHAP (Using XGBoost)
# ---------------------------import pandas as pd
import numpy as np
import os
import re
import shap
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

import pandas as pd
import numpy as np
import os
import re
import shap
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

import pandas as pd
import numpy as np
import os
import re
import shap
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

def train_and_compute_shap(dataset, dataset_name, selected_groups, output_dir='./', additional_exclude_cols=None):
    # Prepare the dataset
    dataset = add_median_lifespan_increase(dataset)
    
    # Filter dataset for selected groups
    dataset = dataset[dataset['Grp_Sex'].isin(selected_groups)].reset_index(drop=True)
    
    # Default columns to exclude
    default_exclude_cols = [
        "median_lifespan_increase", "Grp_Sex", "Lifespan_Increased2", "Grp",
        "Mouse", "ID", "group", "X", "Treatment", "X18198", "Unnamed: 0"
    ]
    
    # Combine exclude lists
    if additional_exclude_cols is not None:
        exclude_cols = default_exclude_cols + additional_exclude_cols
    else:
        exclude_cols = default_exclude_cols

    # Get predictor variables
    predictor_vars_original = [col for col in dataset.columns if col not in exclude_cols]
    
    # Convert predictor variables to numeric
    dataset_numeric = dataset[predictor_vars_original].apply(pd.to_numeric, errors='coerce')
    valid_cols = dataset_numeric.columns[~dataset_numeric.isin([np.nan, np.inf, -np.inf]).any()]
    dataset_numeric = dataset_numeric[valid_cols]
    
    # Remove rows with missing values
    dataset_numeric = dataset_numeric.dropna()
    dataset = dataset.loc[dataset_numeric.index].reset_index(drop=True)
    dataset_numeric = dataset_numeric.reset_index(drop=True)
    
    # Create a mapping from cleaned names to original
    cleaned_names = [clean_column_name(col) for col in dataset_numeric.columns]
    mapping_dict = dict(zip(cleaned_names, dataset_numeric.columns))
    dataset_numeric.columns = cleaned_names  # Rename dataset

    # Train XGBoost Model
   # 'subsample': 0.8, 'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.2, 'colsample_bytree': 0.6},
    xgb_model = XGBRegressor(n_estimators=500,
                             random_state=42,
                             subsample =0.8,
                             learning_rate=0.2,
                             max_depth = 3,
                             colsample_bytree =0.6,
                             n_jobs=-1,
                            deterministic=True)
    xgb_model.fit(dataset_numeric, dataset['median_lifespan_increase'])

    # Compute SHAP values
    explainer = shap.TreeExplainer(xgb_model)
    shap_values = explainer.shap_values(dataset_numeric)

    # **Extract SHAP's Default Sorting**
    mean_abs_shap = np.abs(shap_values).mean(axis=0)  # Mean absolute SHAP values
    shap_default_order = np.argsort(mean_abs_shap)[::-1]  # Sort in descending order

    # Extract ordered feature names **using SHAP's default sorting**
    ordered_cleaned = np.array(dataset_numeric.columns)[shap_default_order]  # Ordered feature names
    ordered_pretty = [mapping_dict.get(col, col) for col in ordered_cleaned]  # Map back to original names

    # **Save the exact SHAP default sorting to a CSV file**
    order_df = pd.DataFrame({'Order': range(1, len(ordered_pretty) + 1), 'Feature': ordered_pretty})
    order_df.to_csv(os.path.join(output_dir, f"SHAP_Default_Feature_Order_{dataset_name}.csv"), index=False)

    # **Force the SHAP summary plot to use this exact order**
    ordered_dataset_numeric = dataset_numeric[ordered_cleaned]
    pretty_feature_names = ordered_pretty  # Ensure SHAP uses the original names in the plot

    # **Save SHAP summary plot using the extracted default sorting**
    plt.figure()
    shap.summary_plot(
        shap_values[:, shap_default_order],  # Use SHAP’s default sorting
        ordered_dataset_numeric,
        feature_names=pretty_feature_names,
        show=False
    )
    plt.title(f"SHAP Summary Plot for {dataset_name}")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"SHAP_Summary_{dataset_name}.png"))
    plt.savefig(os.path.join(output_dir, f"SHAP_Summary_{dataset_name}.pdf"))
    plt.close()

    return order_df  # Return order for debugging if needed


# ---------------------------
# Main Script
# ---------------------------
# Define the groups to analyze
selected_groups = ["M_Cana", "M_Aca", "M_Rapa", "M_17aE2", "M_CR", "M_Cont_12"]

# Specify the output directory for results and SHAP files
output_directory = "../Paper_Figures"
os.makedirs(output_directory, exist_ok=True)

# Initialize a DataFrame to collect all results
all_results = pd.DataFrame()

# Define additional columns to exclude (adjust these as needed)
additional_exclude_cols = ['Mouse', 'mouse', 'Condition', 'Treatment', 'Unmaed:0']

# Assume that 'datasets' is a dictionary of your datasets, for example:
# datasets = {
#     'FM': pd.read_csv('FM.csv'),
#     'AM': pd.read_csv('AM.csv'),
#     ... 
# }
for dataset_name, dataset in datasets.items():
    print(f"\nProcessing dataset: {dataset_name}\n")
    try:
        results = train_and_compute_shap(
            dataset=dataset,
            dataset_name=dataset_name,
            selected_groups=selected_groups,
            output_dir=output_directory,
            additional_exclude_cols=additional_exclude_cols
        )
    
        if results is not None:
            all_results = pd.concat([all_results, results], ignore_index=True)
    except Exception as e:
        print(f"Error processing dataset: {dataset_name}")
        print(f"Error message: {str(e)}")

# Optionally, save the concatenated results
all_results.to_csv(os.path.join(output_directory, "All_Results_Combined.csv"), index=False)



Processing dataset: FM



/home/brettonb/.local/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [18:10:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "deterministic" } are not used.

  warnings.warn(smsg, UserWarning)


Python environment details saved to ../Paper_Figures/Package_Info/pythonStep3_environment.txt


In [13]:
#This broke for some reason. I have no idea why. The code above will work for the paper, this was cool when it was working.
import os
import pandas as pd
from GenerateSHAPFuncs import train_and_compute_shap

# =============================
# 5) Example Usage
# =============================
if __name__ == "__main__":
    # Load data
    AM = pd.read_csv('../Step1_LoadingAndCleaningData/AM.csv')
    FM = pd.read_csv('../Step1_LoadingAndCleaningData/FM.csv')
    AP = pd.read_csv('../Step1_LoadingAndCleaningData/AP.csv')
    FP = pd.read_csv('../Step1_LoadingAndCleaningData/FP.csv')
    AMP = pd.read_csv('../Step1_LoadingAndCleaningData/AMP.csv')
    FMP = pd.read_csv('../Step1_LoadingAndCleaningData/FMP.csv')

    datasets = {
        'FM': FM
     #   'AM': AM,
    #    'AP': AP,
   #     'FP': FP,
  #      'FMP': FMP,
 #       'AMP': AMP
    }

    selected_groups = ["M_Cana", "M_Aca", "M_Rapa", "M_17aE2", "M_CR", "M_Cont_12"]
    output_directory = "ShapOutput"
    os.makedirs(output_directory, exist_ok=True)

    all_results = pd.DataFrame()

    additional_exclude_cols = [
        'Grp_Sex', 'Lifespan_Increased2', 'Grp', 
        'Mouse', 'ID', 'group', 'Treatment', 'X18198', 'Condition', 'Sex', 
        'Mouse_ID', 'Unnamed: 0'
    ]

    for dataset_name, dataset in datasets.items():
        print(f"\nProcessing dataset: {dataset_name}\n")
        try:
            results = train_and_compute_shap(
                dataset=dataset,
                dataset_name=dataset_name,
                selected_groups=selected_groups,
                output_dir=output_directory,
                additional_exclude_cols=additional_exclude_cols
            )
        
            if results is not None:
                all_results = pd.concat([all_results, results], ignore_index=True)
        except Exception as e:
            print(f"Error processing dataset: {dataset_name}")
            print(f"Error message: {str(e)}")

    # Optionally inspect or save all_results
    print("\nFinal concatenated results:")
    print(all_results.head())
    all_results.to_csv(os.path.join(output_directory, "All_Results_Combined.csv"), index=False)



Processing dataset: FM

Number of samples in FM: 94
Number of valid predictor variables: 1051
Groups in dataset: M_Aca, M_Cont_12, M_Cana, M_CR, M_17aE2, M_Rapa
Unique values in median_lifespan_increase:
[22.  0. 14. 32. 12. 23.]
Error processing dataset: FM
Error message: feature_names must be string, and may not contain [, ] or <

Final concatenated results:
Empty DataFrame
Columns: []
Index: []
